# Foundations

### Convert to base 2

In [3]:
def convert(x):
    if x == 0: return "0"
    if x == 1: return "1"
    return convert(x//2) + convert(x%2)

convert(34)

'100010'

### Collatz sequence

In [14]:
def collatz(n):
    print(n, " ", end="")
    
    if n == 1: 
        return
    
    if n % 2 == 0: 
        collatz(n // 2)
        return
    
    collatz(3*n + 1)
    
collatz(7)


7  22  11  34  17  52  26  13  40  20  10  5  16  8  4  2  1  

# Classic example

### Subdivisions of a ruller

In [20]:
def ruler(n):
    if n==0: return " "
    return ruler(n-1) + str(n) + ruler(n-1)        
    
ruler(4)

' 1 2 1 3 1 2 1 4 1 2 1 3 1 2 1 '

### Towers of Hanoi

In [32]:
def reverse(direction):
    if direction=="L": return "R"
    if direction=="R": return "L"
    raise Exception("unknown direction: " + direction)

def hanoi(n, direction):
    """
    Assumes that we have one sorted pile of disks with largest disk size n on the middle spindle.
    The returned instruction describes how disks with different sizes must be moved so that the whole sorted pile
    would relocate in the specified direction. Moves are cyclic, L means Left, R means Right. Cyclic means that 
    if we move leftmost to the left it would cycle through to the rightmost position, simialr for other direction.
    """
    if n==0: return " "
    moveSmallerPileAway = hanoi(n-1, reverse(direction))
    moveLargeDisk = str(n) + direction
    moveSmallerPileOnTop = hanoi(n-1, reverse(direction))
    return moveSmallerPileAway + moveLargeDisk + moveSmallerPileOnTop
    

In [34]:
hanoi(4, "L")

' 1R 2L 1R 3R 1R 2L 1R 4L 1R 2L 1R 3R 1R 2L 1R '

Disk 1 always moves right. Disk 2 always moves left. The same for all other disk sizes - they move only in one direction. And if we always move the smallest disk to the right, there is only one possible move available on the other two disks since we can't put anything on top of the smallest disk. This is the algorithm that can be explained to a human. And this algorithm would work without needing lots of stack and that would cover any n.

And the compute complexity is 2\*\*N - 1. For the profecy time estimate we need to check n for 64. And mutiply by the time it takes to move the disk. Let's assume it is one minute.

In [42]:
operations = 2**64 - 1
inSeconds = operations * 60
inCenturies = inSeconds / 60 / 60 / 24 / 365 / 100
inCenturies

350965450413.0432

# Recursive graphics

## H space filling curve

Like attach electric source to a lot of sites: electrical wiring for a city, electric curcuit chips. Gives connection to everywhere in an organized fashion.

In [1]:
from math import sqrt
import plotly.graph_objects as go
from plotly.graph_objects import Figure as figure
from plotly.graph_objects import Scatter as scatter

In [78]:
def construct(length = 1.0, x0 = 0.0, y0 = 0.0):
    delta = length / 2.0

    # Left vertical
    x1 = [x0 - delta, x0 - delta]
    y1 = [y0 - delta, y0 + delta]

    # Horizontal line
    x2 = [x0 - delta, x0 + delta]
    y2 = [y0, y0]

    # Right vertical
    x3 = [x0 + delta, x0 + delta]
    y3 = [y0 - delta, y0 + delta]
    
    # Combining all together
    x = x1 + [None] + x2 + [None] + x3
    y = y1 + [None] + y2 + [None] + y3
    return x, y

def corners(length = 1.0, x0 = 0.0, y0 = 0.0):
    delta = length / 2.0
    x = [x0 - delta, x0 - delta, x0 + delta, x0 + delta]
    y = [y0 - delta, y0 + delta, y0 - delta, y0 + delta]
    return x, y

def recursive_construct(length = 1.0, x0 = 0.0, y0 = 0.0, n = 0):
    if n==0: return [], []
    
    xc, yc = corners(length, x0, y0)
    x1, y1 = recursive_construct(length / 2.0, xc[0], yc[0], n-1)
    x2, y2 = recursive_construct(length / 2.0, xc[1], yc[1], n-1)
    x3, y3 = recursive_construct(length / 2.0, xc[2], yc[2], n-1)
    x4, y4 = recursive_construct(length / 2.0, xc[3], yc[3], n-1)

    x,y = construct(length, x0, y0)
    x = x + [None] + x1 + [None] + x2 + [None] + x3 + [None] + x4
    y = y + [None] + y1 + [None] + y2 + [None] + y3 + [None] + y4
        
    return x, y

def draw(x, y):   
    f = figure(
        scatter(
            x = x,
            y = y,
        ),
        layout = {
            "template": "plotly_white",
            "yaxis": {"scaleanchor": "x", "scaleratio": 1},
        },  
    )
    f.show()
    
x,y = recursive_construct(length = 1.0, x0 = 0.0, y0 = 0.0, n = 5)
draw(x,y)


In [148]:
def make_h(d=0.5, x=0, y=0):
    return [
        (x-d, y-d, x-d, y+d),
        (x-d, y+0, x+d, y+0),
        (x+d, y-d, x+d, y+d),
    ]


def make_h_recurse(n, sz = 1, x = 0, y = 0):
    if n <= 0: raise ValueError

    d = sz / 2  
    result = make_h(d, x, y)

    if n > 1:
        result += make_h_recurse(n-1, d, x-d, y-d)
        result += make_h_recurse(n-1, d, x-d, y+d)
        result += make_h_recurse(n-1, d, x+d, y-d)
        result += make_h_recurse(n-1, d, x+d, y+d)

    return result


def convert_lines_to_xy(lines):
    "lines is array of tuples (x0, y0, x1, y1)"
    x_lines = [[line[0], line[2], None] for line in lines]
    y_lines = [[line[1], line[3], None] for line in lines]
    x = [x for x_line in x_lines for x in x_line]
    y = [y for y_line in y_lines for y in y_line]
    return x[:-1], y[:-1]
    

def draw(lines):    
    x, y = convert_lines_to_xy(lines)
    figure(
        scatter(x = x, y = y),
        layout = {
            "template": "plotly_white",
            "yaxis": {"scaleanchor": "x", "scaleratio": 1},
        },  
    ).show()   

    
lines = make_h_recurse(4)
draw(lines)


## Fractional Brownian motion

Actual name of it is *midpoint displacement method*. Can be used as a model for:
- stock prices
- dispersion iof fluids
- shapes of mointains and clouds

In [149]:
from random import gauss
import plotly.graph_objects as go
from plotly.graph_objects import Figure as figure
from plotly.graph_objects import Scatter as scatter 

In [179]:
def brownian_middle(x0, y0, x1, y1, mu = 0, sigma = 1):
    xm = (x0 + x1) / 2
    ym = (y0 + y1) / 2
    d = gauss(mu, sigma)
    return xm, ym + d

def midpoint_displacement(x0, y0, x1, y1, mu = 0, sigma = 1, n = 0):
    if n==0: return [], []
    
    if n==1:
        xm, ym = brownian_middle(x0, y0, x1, y1, mu, sigma)
        return [x0,xm,x1], [x0,xm,x1]
    
   
print(brownian_middle(0,0, 1,0))
print(midpoint_displacement(0,0, 1,0, n=1))
    
    

(0.5, 0.15604552246661413)
([0, 0.5, 1], [0, 0.5, 1])


In [166]:
gauss?